# 모델링 프로젝트(SVM)  
## 만들 모델: 어떤 문장이 악플인지 아닌지를 “분류”하는 모델 만들기(SVM, Bert)  
주제 선택 배경: sns 상에서 악플로 인한 피해가 심각  
모델 적용 방식: 에브리타임 등의 sns에서 악플을 긁어오기 -> 만든 모델에 적용시켜 악플 분류하기  

더 나아가서:  
1) 감성사전 이용해 악플로 분류된 문장에서 부정 키워드 뽑아내기 -> 긍정 단어로 순화시키기  
2) 에브리타임 게시판 별로 악플 비율 비교해보기  

---

## SVM을 선택한 이유  
1. bert는 딥러닝 모델이기 때문에 머신러닝 모델인 SVM 선택  
2. SVM은 범주나 수치 예측에 사용 가능-> 주로 분류 목적으로 사용됨.  
https://muzukphysics.tistory.com/135
3. 과적합 되는 경우가 적고 신경망보다 사용하기 쉽다.   

---

## SVM 활용 시에 유의사항
1. 여러 조합 테스트 필요(커널과 모델에서)    
2. 여러 연산이 필요, 입력 데이터 셋이 많으면 학습 속도가 느림.  
3. 해석 힘듦./블랙박스 형태(해석이 불가능한 것은 아님.)  

## 1. load dataset   
https://github.com/kocohub/korean-hate-speech  
https://www.kaggle.com/c/korean-hate-speech-detection/overview  
구성: 1) labeled 2) unlabeled and 3) news_title  
### 1) labeled  
    9381 comments-수동 분류(7896 traing set, 471 validation set, 974 test set)  
    social bias가 있나, hate speech인가로 labeled됨.  
    - contain_gender_bias: True, False  
    - social bias: gender, others, none   
    - hate speech: hate, offensive, none   
### 2) unlabeled  
    2,033,893 comments  
    can be used in various ways: pretraining language model, semi-supervised learning 등  
### 3) news_title  
    comments 이해에 도움  

In [ ]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
!pip install koco

  Created wheel for koco: filename=koco-0.2.3-cp37-none-any.whl size=7363 sha256=e5153e5952723e64395ecef74883a667b8167df1f88919d0533d0a3883b88c10
  Stored in directory: /root/.cache/pip/wheels/bd/92/69/283e258d3f59e4062561e47fb3598cbcbf103ad7c181af3bce
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=d869ecafe7b5785cf3e223e82ddb2f0bfb97b05825761eb3b5eb700294640968
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built koco wget


In [ ]:
import koco

**머신러닝에서의 data set**  
1) train: 학습  
2) dev(validation): 최적의 모델을 찾아감.  
3) test: 성능을 평가  
test set은 모델을 평가할 때만 사용함.  
모델을 학습할 때는 train과 dev으로 나뉘어 cross over validation을 진행함.  
cross over validation: dev set을 데이터의 일정 부분만큼 떼내고, 나머지 데이터로 train을 진행하고, 일정 step마다 dev set로 모델을 test하며 가장 dev set의 성능을 좋게 할 때 끊는(early stopping) 방법을 모든 데이터에 대해서 반복하는 것을 말한다.  
https://tekhartha.com/machine%20learning/2019/03/24/data_dividing/

### labeled: train_dev

In [ ]:
# train_dev: train set, dev set
train_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')

In [ ]:
type(train_dev)

dict

In [ ]:
train_dev.keys()

dict_keys(['train', 'dev'])

In [ ]:
type(train_dev['train'])

list

In [ ]:
# train_dev의 형태
train_dev['train'][0]

{'bias': 'others',
 'comments': '(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....',
 'contain_gender_bias': False,
 'hate': 'hate',
 'news_title': '"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"'}

In [ ]:
print(len(train_dev['train']), len(train_dev['dev']))

7896 471


### labeled: test

In [ ]:
test = koco.load_dataset('korean-hate-speech', mode='test')

In [ ]:
type(test)

list

In [ ]:
test[0]

{'comments': 'ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ',
 'news_title': '"샤이니 온유, 클럽 강제추행 \'무혐의\' 처분 받았다"'}

In [ ]:
len(test)

974

### unlabeled

In [ ]:
unlabeled = koco.load_dataset('korean-hate-speech', mode='unlabeled')

In [ ]:
type(unlabeled)

list

In [ ]:
unlabeled[0]

{'comments': '지드래곤은 난봉꾼이란...댓글도 달렸네 ㅋㅋ 이주연 학창시절 사진 보고 와라. 요즘 웬만한 여자 연예인하고 붙여놔도....미모가 최고였단다.ㅋ 5대 얼짱 출신.',
 'news_title': '"[단독] 지드래곤♥이주연, 제주도 데이트…2018년 1호 커플 탄생"'}

## 2. 데이터셋 처리  



In [ ]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC

import warnings
warnings.filterwarnings(action='ignore')

### idea) hate 라벨에서 hate와 offensive를 묶어 1로, none을 0으로 바꾸기
=> 이렇게 나누었을 때 가장 높은 성능을 얻을 수 있었음.


In [ ]:
train_df = pd.DataFrame(train_dev['train'], columns=['comments', 'hate'])
train_df

,comments,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate
...,...,...
7891,힘내세요~ 응원합니다!!,none
7892,힘내세요~~삼가 고인의 명복을 빕니다..,none
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,none
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none


In [ ]:
train_df['hate'].unique()

array(['hate', 'none', 'offensive'], dtype=object)

In [ ]:
# hate와 offensive를 1로,none을 0으로
hate_change = {'hate':1, 'offensive':1, 'none':0}
train_df = train_df.replace({'hate': hate_change})
train_df

,comments,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,1
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",1
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1
...,...,...
7891,힘내세요~ 응원합니다!!,0
7892,힘내세요~~삼가 고인의 명복을 빕니다..,0
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,0
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,0


In [ ]:
train_df.isnull().sum()
# 결측치 없음.

comments    0
hate        0
dtype: int64

In [ ]:
train_df.groupby('hate').count()

,comments
hate,
0,3486
1,4410


### mecab으로 comments를 토큰화
참고) http://textmining.kr/?p=418  
https://wikidocs.net/21693

In [ ]:
# !pip install konlpy

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [ ]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:
! bash install_mecab-ko_on_colab190912.sh


Installing konlpy.....
     |████████████████████████████████| 19.4MB 35.2MB/s 
     |████████████████████████████████| 460kB 40.3MB/s 
     |████████████████████████████████| 92kB 8.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-03-17 12:45:12--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c0:3470, 2406:da00:ff00::22c2:513, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=eorRWxGqSxK%2FgmWxMQpzERdKXYM%3D&Expires=1615986282&AWSAccessKeyId=AKIA6KOSE3B

In [ ]:
from konlpy.tag import Mecab
import re

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
def hangul(text):
    # 한글과 띄어쓰기를 제외한 모든 글자
    hangul = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]+")
    result = hangul.sub(" ", text)
    return result

def get_noun_verb_adjective(text):
    text = hangul(text)
    mecab = Mecab()
    temp_X = mecab.pos(text) # 품사 붙여주기
    token = []
    for idx, word in enumerate(temp_X):
        if (word[1] in ["NNG","NNP", 'VV', 'VA', 'VCP', 'VCN', 'IC', 'MM', "MAG"]) and (word[1] not in stopwords): # 명사, 동사, 형용사 이외에도 중요해보이는 품사 포함
            token.append(word[0])
    return token

In [ ]:
print(get_noun_verb_adjective("내가 제일 좋아하는 건 여름 그 맛."))
get_noun_verb_adjective("소년과 두더지와 여우와 말!!!//")

['제일', '좋', '하', '여름', '그', '맛']


['소년', '두더지', '여우', '말']

In [ ]:
# 각 명사, 동사, 형용사가 몇 번 나왔는지 수를 세는 과정
# cv = CountVectorizer(tokenizer= get_noun_verb_adjective)
# tdm = cv.fit_transform(train_df["comments"])
# # cv.vocabulary_
# # len(cv.vocabulary_)
    

## 3. SVM 모델

### gridsearch  
https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.YFGokK8zZPa

In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

# # 파이프라인
# # 데이터 사전 처리 및 분류의 모든 단계를 포함하는 단일 개체를 만들 수 있다.(한 번에 진행할 수 있어 편하다.)

# text_svm = make_pipeline(CountVectorizer(tokenizer=get_noun_verb_adjective, min_df=3), 
#                           TfidfTransformer(), 
#                          SVC(random_state=1))

# param_range = [0.01, 0.1, 1, 10.0, 100.0] 
# gamma_range = [0.1, 1, 10]
# param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']}, {'svc__C': param_range, 'svc__gamma': gamma_range, 'svc__kernel': ['rbf']}]

# print('gridsearchcv')
# gs = GridSearchCV(estimator=text_svm, 
#                   param_grid=param_grid, 
#                   scoring=make_scorer(accuracy_score), 
#                   cv=5,
#                   n_jobs=-1)
#  # accuracy로 grid search를 한 이유는 f1_score로 하니까 오히려 0을 잘 예측 못하는 문제가 있었음.


In [ ]:
# print('fit')
# gs = gs.fit(train_df["comments"], train_df["hate"])

# print('final params', gs.best_params_)   # 최적의 파라미터 값 출력
# print('best score', gs.best_score_)      # 최고의 점수

### 최고의 파라미터로 text_clf_svm 모델을 적합

In [ ]:
# # 최고의 파라미터로 text_clf_svm 모델을 적합
# params = gs.best_params_
# C_ = params['svc__C'] 
# kernel_ = params['svc__kernel']
# gamma_ = params['svc__gamma']
# text_clf_svm = make_pipeline(CountVectorizer(tokenizer=get_noun_verb_adjective, min_df = 3), #각 텍스트에서 단어 출현 횟수를 카운팅한 벡터(TF 방식)
#                             TfidfTransformer(), #변환기: TFidfVectorizer가 아님.
#                             SVC(C = C_, kernel = kernel_, gamma = gamma_))
# text_clf_svm.fit(train_df["comments"], train_df["hate"])

In [ ]:
text_clf_svm = make_pipeline(CountVectorizer(tokenizer=get_noun_verb_adjective, min_df=3), #각 텍스트에서 단어 출현 횟수를 카운팅한 벡터(TF 방식)
                            TfidfTransformer(), #변환기: TFidfVectorizer가 아님.
                            SVC(C = 1, kernel = 'rbf', gamma = 1))
text_clf_svm.fit(train_df["comments"], train_df["hate"])

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=3,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<functio...
                                 vocabulary=None)),
                ('tfidftransformer',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('svc',
                 SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
             

### dev set으로 정확도 평가  
undersampling을 해서 평가하기

In [ ]:
dev_df = pd.DataFrame(train_dev['dev'], columns=['comments', 'hate'])

# hate와 offensive는 1로, none은 0으로
hate_change = {'hate':1, 'offensive':1, 'none':0}
dev_df = dev_df.replace({'hate': hate_change})

In [ ]:
dev_df.head()

,comments,hate
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,0
1,지현우 나쁜놈,1
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,1
3,설마 ㅈ 현정 작가 아니지??,1
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,1


### undersampling해서 정확도 평가  

In [ ]:
#데이터 추출, 개수 확인

shrink0 = dev_df[dev_df['hate']==0]

shrink1 = dev_df[dev_df['hate']==1][:160]

print(len(shrink0), len(shrink1))

#합치고 랜덤으로 섞은 뒤 train_df에 다시 저장
dev_fixed = pd.concat([shrink0, shrink1])

dev_fixed_df = dev_fixed.sample(frac=1).reset_index(drop=True)


160 160


In [ ]:
dev_fixed_df.head()

,comments,hate
0,82년쯔음 여자들이 딱 중고교때부터 불평불만이 많았음,1
1,형 화이팅,0
2,여자 두명이면 서로 까고 세명이면 둘이 하나깜,1
3,유이 자연스러워진 연기,0
4,"차화연씨가 언급한것도 아니고, 조세호가 질문해서 거기에 대답한건데 내용도 잘 모르는...",1


In [ ]:
# 성능 평가: accuracy
from sklearn.metrics import accuracy_score
predicted_svm = text_clf_svm.predict(dev_fixed_df['comments'])
score = accuracy_score(dev_fixed_df['hate'], predicted_svm)
print(score)

# 성능평가: f1_score
from sklearn.metrics import f1_score
score = f1_score(dev_fixed_df['hate'], predicted_svm)
print(score)

# confustion matrix
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(dev_fixed_df['hate'], predicted_svm)
mat

0.75625
0.7621951219512195


array([[117,  43],
       [ 35, 125]])

In [ ]:
# 그냥 시험해보기
text_clf_svm.predict(["헐 김철 미애가 자갈 밟고 휘청거리니까 다 치워놓음진짜 미친거니?", 
                      "?! 김철이 자갈 치워준 거라구요??? 미쳤다 애송이 주제에 로맨스를 아는구나 너무 좋아서 이마 치다가 머리 깨짐 ㅠㅠㅠ 어른들 놀려서 왁왁거릴 때도 저랬는데 미애한테 꽂히면 얼마나 더 로맨티스트라는 거야 ㅠㅠㅠ",
                      "아니 근데 왜 저런 기레기들은 고소가 안돼??????? 저것도 스토킹이고 사생활 침해 아님?????? 범죄를 하고있는거 아니야????????????",
                      "준나 “디스패치” 같네요 “디스패치” 야 발 롬 들;",
                      "기자 개패고싶다",
                      "아가리를 그냥 확!",
                      "넌 정말 예뻐",
                      "너와 친구여서 행복해"])
# https://comic.naver.com/webtoon/detail.nhn?titleId=761722&no=10&weekday=fri 
# https://comic.naver.com/webtoon/detail.nhn?titleId=709731&no=127&weekday=mon                    

array([0, 1, 1, 1, 0, 1, 0, 0])

In [ ]:
## dev_df로 성능 평가

# 성능 평가: accuracy
from sklearn.metrics import accuracy_score
predicted_svm = text_clf_svm.predict(dev_df['comments'])
score = accuracy_score(dev_df['hate'], predicted_svm)
print(score)

# 성능평가: f1_score
from sklearn.metrics import f1_score
score = f1_score(dev_df['hate'], predicted_svm)
print(score)

# confustion matrix
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(dev_df['hate'], predicted_svm)
mat

0.7600849256900213
0.8100840336134455


array([[117,  43],
       [ 70, 241]])

**결과**  
undersampling했을 때 
**f1_score를 기준으로**  
{'svc__C': 1, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}  
accuracy: 0.675  
f1: 0.7386934673366835
confustion matrix:  
array([[ 69,  91],  
[ 13, 147]])  
-> 별로임.  

C 너무 작으면 0을 잘 분류하지 못함. C를 어느 정도 키워서 f1 score 떨어뜨리더라도 0도 잘 분류하게끔.->grid search를 다시  
`C = 20, gamma = 0.1`  
0.734375  
0.7368421052631579  
array([[116,  44],  
       [ 41, 119]])  

`C = 10, gamma = 0.1`  
0.7375  
0.7439024390243903  
array([[114,  46],  
       [ 38, 122]])  

`C = 5, gamma = 0.1`  
0.734375  
0.743202416918429  
array([[112,  48],  
       [ 37, 123]])  

`C = 3, gamma = 0.1`  
0.715625  
0.736231884057971  
array([[102,  58],  
       [ 33, 127]])  


C 낮추면 1을 분류 잘 하고, 0은 분류 잘 못 하지만 C 높이면 0을 더 분류 잘 하게 됨.  

**accuracy를 기준으로**  
`{'svc__C': 1, 'svc__gamma': 1, 'svc__kernel': 'rbf'}`  
0.75  
0.7633136094674556  
array([[111,  49],  
       [ 31, 129]])

dev_df 전체:  
0.7558386411889597  
0.8099173553719008  
array([[111,  49],  
       [ 66, 245]])



## 4. test set에 적용시켜보기

In [ ]:
test_comments = pd.DataFrame(test, columns = ['comments'])
test_comments.head()

,comments
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ
1,둘다 넘 좋다~행복하세요
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요
4,장현승 얘도 참 이젠 짠하다...


In [ ]:
predicted_test = text_clf_svm.predict(test_comments['comments'])
predicted_test[:5]

array([0, 0, 1, 0, 0])

In [ ]:
test_comments['hate_predict'] = predicted_test
test_comments.head(5)

,comments,hate_predict
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,1
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,0


## 5. 에브리타임 게시판 별로 악성 게시물 비율 확인해보기

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
def get_hate_ratio(data):
    df = pd.read_csv('/gdrive/My Drive/everytime_data/{}.csv'.format(data), sep = ",")
    df = df.rename({'0': 'comments'}, axis = 'columns')
    predicted = text_clf_svm.predict(df['comments'])
    df['hate_predict'] = predicted

    ratio = df.groupby('hate_predict').count()
    hate_ratio = ratio['comments'][1] / (ratio['comments'][0] + ratio['comments'][1])
    return ratio, hate_ratio

### 1) 정보게시판

In [ ]:
info_df = pd.read_csv('/gdrive/My Drive/everytime_data/정보게시판.csv', sep = ",")

In [ ]:
info_df = info_df.rename({'0': 'comments'}, axis = 'columns')

In [ ]:
info_df.head()

,comments
0,요일: 월~ 목 시간: 18~23시 장소: 씨유 연세 암병원점 다른 분과 번갈아 가...
1,1. 플레이스토어/앱스토어에서 코스모스라고 검색해서 첫 번째 사진에 나오는 앱을 깔...
2,"안녕하세요, YDEC에서 근무 중인 학부생 인턴입니다. 지난 번에 정보게시판을 통해..."
3,안녕하세요 책갈피입니다. 21-1학기 사물함 추가 신청을 받습니다. (이미 배정받으...
4,"안녕하세요, YDEC에서 근무중인 학부생인턴입니다. 오늘 많은 분들이 문의 주셨던 ..."


In [ ]:
info_predict = text_clf_svm.predict(info_df['comments'])
info_df['hate_predict'] = info_predict

info_df.groupby('hate_predict').count()

,comments
hate_predict,
0,611
1,416


In [ ]:
info_ratio = info_df.groupby('hate_predict').count()
info_hate = info_ratio['comments'][1] / (info_ratio['comments'][0] + info_ratio['comments'][1])
info_hate

0.4050632911392405

In [ ]:
info_ratio, info_hate = get_hate_ratio('정보게시판')
print(info_ratio)
print(info_hate)

              comments
hate_predict          
0                  611
1                  416
0.4050632911392405


### 2) 새내기 게시판

In [ ]:
freshman_ratio, freshman_hate = get_hate_ratio('새내기 게시판')
print(freshman_ratio)
print(freshman_hate)

              comments
hate_predict          
0                 1496
1                 2304
0.6063157894736843


### 3) 시사, 이슈 게시판

In [ ]:
issue_ratio, issue_hate = get_hate_ratio('시사, 이슈 게시판')
print(issue_ratio)
print(issue_hate)

              comments
hate_predict          
0                  135
1                  208
0.6064139941690962


### 4) 핫게시판

In [ ]:
hot_ratio, hot_hate = get_hate_ratio('핫게시판')
print(hot_ratio)
print(hot_hate)

              comments
hate_predict          
0                 1269
1                 2550
0.6677140612725845


### 5) 자유게시판본문

In [ ]:
free_ratio, free_hate = get_hate_ratio('자유게시판본문')
print(free_ratio)
print(free_hate)

              comments
hate_predict          
0                  799
1                 1025
0.5619517543859649


### 6) 비밀게시판 통합

In [ ]:
secret_ratio, secret_hate = get_hate_ratio('비밀게시판 통합')
print(secret_ratio)
print(secret_hate)

              comments
hate_predict          
0                 1269
1                 2550
0.6677140612725845
